In [1]:
import sys
from pathlib import Path
from sampler_perturbations import f_theta_sampler
import numpy as np
import jax
import jax.numpy as jnp
from jax import random, jit, vmap
from tqdm.auto import tqdm
Path("results").mkdir(exist_ok=True)
%load_ext autoreload
%autoreload 2

In [2]:
from all_tests import hsicfuse_test, hsic_test
from all_tests import hsicagginc_test, hsicagg_test
from all_tests import nfsic_test, nyhsic_test, fhsic_test

### Vary difficulty (d=2, rep=200, N=500)

In [3]:
repetitions          = 200
scales               = (0.0, 0.2, 0.4, 0.6, 0.8, 1.0)
sample_size          = 500 
d                    = 2
f_theta_seed         = 0
p                    = 1
s                    = 1

tests = (hsicfuse_test, hsic_test, hsicagginc_test, hsicagg_test, nfsic_test, nyhsic_test, fhsic_test)
outputs = jnp.zeros((len(tests), len(scales), repetitions))
outputs = outputs.tolist()
key = random.PRNGKey(42)
seed = 42
for s in tqdm(range(len(scales))):
    scale = scales[s]
    for i in tqdm(range(repetitions)):
        key, subkey = random.split(key)
        perturbation_multiplier = np.exp(d) * p ** s * scale 
        Z = f_theta_sampler(f_theta_seed, seed, sample_size, p, s, perturbation_multiplier, d)
        X = np.expand_dims(Z[:, 0], 1)
        Y = np.expand_dims(Z[:, 1], 1)
        key, subkey = random.split(key)
        seed += 1
        for t in range(len(tests)):
            test = tests[t]
            outputs[t][s][i] = test(X, Y, subkey, seed)

output = jnp.mean(jnp.array(outputs), -1) # the last dimension is eliminated

jnp.save("results/perturbations_vary_dif.npy", output)
jnp.save("results/perturbations_vary_dif_x_axis.npy", scales)

print("scales :", scales)
print("sample size :", sample_size)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(output[t])

print(" ")
print(output)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

scales : (0.0, 0.2, 0.4, 0.6, 0.8, 1.0)
sample size : 500
 
<function hsic_test at 0x7f691c68eca0>
[0.045      0.235      0.90999997 1.         1.         1.        ]
 
[[0.045      0.235      0.90999997 1.         1.         1.        ]]


## Vary sample size (d=2, rep=200, scale=0.6)

In [6]:
repetitions          = 200
scale                = 0.6
number_perturbations = 2
sample_sizes         = (200, 400, 600, 800, 1000, 1500, 2000, 2500, 3000)
d                    = 2
f_theta_seed         = 0
p                    = 1
s                    = 1

tests = (hsicfuse_test, hsic_test, hsicagginc_test, hsicagg_test, nfsic_test, nyhsic_test, fhsic_test)
outputs = jnp.zeros((len(tests), len(sample_sizes), repetitions))
outputs = outputs.tolist()
key = random.PRNGKey(42)
seed = 42
for s in tqdm(range(len(sample_sizes))):
    sample_size = sample_sizes[s]
    for i in tqdm(range(repetitions)):
        key, subkey = random.split(key)
        perturbation_multiplier = np.exp(d) * p ** s * scale 
        Z = f_theta_sampler(f_theta_seed, seed, sample_size, p, s, perturbation_multiplier, d)
        X = np.expand_dims(Z[:, 0], 1)
        Y = np.expand_dims(Z[:, 1], 1)
        key, subkey = random.split(key)
        seed += 1
        for t in range(len(tests)):
            test = tests[t]
            outputs[t][s][i] = test(X, Y, subkey, seed)

output = jnp.mean(jnp.array(outputs), -1)

jnp.save("results/perturbations_vary_n.npy", output)
jnp.save("results/perturbations_vary_n_x_axis.npy", sample_sizes)

print("sample sizes :", sample_sizes)
print("scale :", scale)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(output[t])

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

sample sizes : (200, 400, 600, 800, 1000, 1500, 2000, 2500, 3000)
scale : 0.6
 
<function hsic_test at 0x7f691c68eca0>
[0.895      0.98999995 1.         1.         1.         1.
 1.         1.         1.        ]


### Vary d (N=500, rep=200, scale=1.0)

In [5]:
repetitions          = 200
scale                = 1.0
number_perturbations = 2
sample_size          = 500
d_values             = (1, 2, 3, 4)
f_theta_seed         = 0
p                    = 1
s                    = 1

tests = (hsicfuse_test, hsic_test, hsicagginc_test, hsicagg_test, nfsic_test, nyhsic_test, fhsic_test)
outputs = jnp.zeros((len(tests), len(d_values), repetitions))
outputs = outputs.tolist()
key = random.PRNGKey(42)
seed = 42
for s in tqdm(range(len(d_values))):
    d = d_values[s]
    for i in tqdm(range(repetitions)):
        key, subkey = random.split(key)
        perturbation_multiplier = np.exp(d + 1) * p ** s * scale 
        Z = f_theta_sampler(f_theta_seed, seed, sample_size, p, s, perturbation_multiplier, d + 1)
        X = Z[:, :1]
        Y = Z[:, 1:]       
        key, subkey = random.split(key)
        seed += 1
        for t in range(len(tests)):
            test = tests[t]
            outputs[t][s][i] = test(X, Y, subkey, seed)

output = jnp.mean(jnp.array(outputs), -1)

jnp.save("results/perturbations_vary_d.npy", output)
jnp.save("results/perturbations_vary_d_x_axis.npy", d_values)

print("d :", d_values)
print("sample size :", sample_size)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(output[t])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

d : (1, 2, 3, 4)
sample size : 500
 
<function hsic_test at 0x7f691c68eca0>
[1.         0.53499997 0.065      0.05      ]
